In [1]:
# Import relevant packages

import pandas as pd

In [2]:
# General class definition
class thermodynamics():
    'This class contains thermodynamic data'
    def __init__(self):
        
        # Dati delle tabelle di Zanardi ref[1]
        
        self.M  = []
        self.A  = []
        self.B  = []
        self.C  = []
        self.I  = []
        self.h  = []
        self.Ov = []
        self.Or = []
        self.Ds = []

    
    def update(self):
        'Updates thermodynamic variable R'
        self.R = 8314/self.M
        
        
    def e_tr(self,T):
        'Computes translational temperature'
        self.e_tr = 3/2 * self.R * T
        return self.e_tr
        

        
    
class specie():
    'This class defines a specie'
    def __init__(self):
        #self.data = []
        self.thermo = thermodynamics()
        self.electro = pd.DataFrame({'Level': [],
                                     'g'    : [],
                                     'Oe'   : []
                                    })

        


In [3]:
#N2 definition

N2 = specie()

N2.thermo.M  = 28.0134
N2.thermo.A  = 2.68e-2
N2.thermo.B  = 3.18e-1
N2.thermo.C  = -1.13e1
N2.thermo.I  = 5.36498e7
N2.thermo.h  = 0
N2.thermo.Ov = 3395
N2.thermo.Or = 2.9
N2.thermo.Ds = 3.36e7
N2.thermo.update()

N2.electro = {'Level': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 
              'g'    : [1, 3, 6, 6, 3, 1, 2, 2, 5, 1, 6, 6, 10, 6, 6],
           'Oe': [0, 7.2231565e+04, 8.5778626e+04, 8.6050267e+04, 
                  9.5351186e+04, 9.8056357e+04, 9.9682677e+04, 
                  1.0489765e+05, 1.1164896e+05, 1.2258365e+05,
                  1.2488569e+05, 1.2824762e+05, 1.3380609e+05, 
                  1.4042964e+05, 1.5049589e+05]
          }
N2.electro = pd.DataFrame(N2.electro)

N2.electro # Print electro table in pandas format


,Level,g,Oe
0,0,1,0.000
1,1,3,72231.565
2,2,6,85778.626
3,3,6,86050.267
4,4,3,95351.186
5,5,1,98056.357
6,6,2,99682.677
7,7,2,104897.650
8,8,5,111648.960
9,9,1,122583.650


In [4]:
#O2 definition

O2 = specie()

O2.thermo.M  = 31.9988
O2.thermo.A  = 4.49e-02
O2.thermo.B  = -8.26e-02
O2.thermo.C  = -9.20
O2.thermo.I  = 3.63832e+07
O2.thermo.h  = 0
O2.thermo.Ov = 2239
O2.thermo.Or = 2.1
O2.thermo.Ds = 1.54e+07
O2.thermo.update()

O2.electro = {'Level': [0, 1, 2, 3, 4, 5, 6], 
           'g': [3, 2, 1, 1, 6, 3, 3],
           'Oe': [0, 1.1391560e+04, 1.8984739e+04,
                 4.7559736e+04, 4.9912421e+04, 5.0922686e+04,
                 7.1898633e+04]
          }
O2.electro = pd.DataFrame(N2.electro)

O2.electro # Print electro table in pandas format


,Level,g,Oe
0,0,1,0.000
1,1,3,72231.565
2,2,6,85778.626
3,3,6,86050.267
4,4,3,95351.186
5,5,1,98056.357
6,6,2,99682.677
7,7,2,104897.650
8,8,5,111648.960
9,9,1,122583.650


In [5]:
#NO definition

NO = specie()

NO.thermo.M  = 30.0061
NO.thermo.A  = 4.36e-02
NO.thermo.B  = -3.36e-02
NO.thermo.C  = -9.58
NO.thermo.I  = 2.97808e+07
NO.thermo.h  = 3.0091e+06
NO.thermo.Ov = 2817
NO.thermo.Or = 2.5
NO.thermo.Ds = 2.09e+07
NO.thermo.update()

NO.electro = {'Level': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 
           'g': [4, 8, 2, 4, 4, 4, 4, 2, 4, 2, 4, 4, 2, 2, 2, 4],
           'Oe': [0, 5.4673458e+04, 6.3171396e+04,
                 6.5994503e+04, 6.9061210e+04, 7.0499985e+04,
                 7.4910550e+04, 7.6288753e+04, 8.6761885e+04,
                 8.7144312e+04, 8.8860771e+04, 8.9817556e+04,
                 8.9884459e+04, 9.0427021e+04, 9.0642838e+04,
                 9.1117633e+04]
          }
NO.electro = pd.DataFrame(NO.electro)

NO.electro # Print electro table in pandas format


,Level,g,Oe
0,0,4,0.000
1,1,8,54673.458
2,2,2,63171.396
3,3,4,65994.503
4,4,4,69061.210
5,5,4,70499.985
6,6,4,74910.550
7,7,2,76288.753
8,8,4,86761.885
9,9,2,87144.312


In [6]:
#N definition

N = specie()

N.thermo.M  = 14.0067
N.thermo.A  = 1.16e-02
N.thermo.B  = 6.03e-01
N.thermo.C  = -1.24e+01
N.thermo.I  = 1.00090e+08
N.thermo.h  = 3.3747e+07
N.thermo.Ov = 0
N.thermo.Or = 0
N.thermo.Ds = 0
N.thermo.update()

N.electro = {'Level': [0, 1, 2], 
           'g': [4, 10, 6],
           'Oe': [0, 2.7664696e+04, 4.1493093e+04]
          }
N.electro = pd.DataFrame(N.electro)

N.electro # Print electro table in pandas format


,Level,g,Oe
0,0,4,0.000
1,1,10,27664.696
2,2,6,41493.093


In [7]:
#O definition

O = specie()

O.thermo.M  = 15.9994
O.thermo.A  = 2.03e-02
O.thermo.B  = 4.29e-01
O.thermo.C  = -1.16e+01
O.thermo.I  = 8.21013e+07
O.thermo.h  = 1.5574e+07
O.thermo.Ov = 0
O.thermo.Or = 0
O.thermo.Ds = 0
O.thermo.update()

O.electro = {'Level': [0, 1, 2, 3, 4], 
           'g': [5, 3, 1, 5, 1],
           'Oe': [0, 2.2770776e+02, 3.2656888e+02,
                  2.2830286e+04, 4.8619930e+04]
          }
O.electro = pd.DataFrame(O.electro)

O.electro # Print electro table in pandas format


,Level,g,Oe
0,0,5,0.00000
1,1,3,227.70776
2,2,1,326.56888
3,3,5,22830.28600
4,4,1,48619.93000


In [8]:
#N2p definition

N2p = specie

N2p.thermo.M  = 28.0128514
N2p.thermo.A  = 2.68e-02
N2p.thermo.B  = 3.18e-01
N2p.thermo.C  = -1.13e+01
N2p.thermo.I  = 0
N2p.thermo.h  = 5.3886e+07
N2p.thermo.Ov = 3395
N2p.thermo.Or = 2.9
N2p.thermo.Ds = 3.00e+07

N2p.electro = {'Level': [0, 1, 2, 3, 4,5 ,6 ,7 ,8 ,9 ,10, 11, 12, 13, 14, 15, 16], 
           'g': [2, 4, 2, 4,8, 8, 4, 4, 4, 4, 8, 8, 4, 4, 2, 2, 4],
           'Oe': [0, 1.3189972e+04, 3.6633231e+04,
                  3.6688768e+04, 5.9853048e+04, 6.6183659e+04,
                  7.5989919e+04, 7.6255086e+04, 8.2010186e+04, 
                  8.4168349e+04, 8.6326512e+04, 8.9204062e+04,
                  9.2081613e+04, 9.2225490e+04, 9.2937684e+04, 
                  9.6397938e+04, 1.0359181e+05]
          }
N2p.electro = pd.DataFrame(N2p.electro)

N2p.electro # Print electro table in pandas format


AttributeError: type object 'specie' has no attribute 'thermo'

In [ ]:
#O2p definition

O2p = specie()

O2p.thermo.M  = 31.9982514
O2p.thermo.A  = 4.49e-02
O2p.thermo.B  = -8.26e-02
O2p.thermo.C  = -9.20
O2p.thermo.I  = 0
O2p.thermo.h  = 3.6592e+07
O2p.thermo.Ov = 2239
O2p.thermo.Or = 2.1
O2p.thermo.Ds = 2.01e+07
O2p.thermo.update()

O2p.electro = {'Level': [0, 1, 2, 3, 4,5 ,6 ,7 ,8 ,9 ,10, 11, 12, 13, 14], 
           'g': [4, 8, 4, 6, 4, 2, 4, 4, 4, 4, 8, 4, 2, 2, 4],
           'Oe': [0, 4.7354408e+04, 5.8373987e+04, 
                 5.8414273e+04, 6.2298966e+04, 6.7334679e+04,
                 7.1219372e+04, 7.6542841e+04, 8.8196920e+04,
                 8.8196920e+04, 9.4239776e+04, 9.4959163e+04,
                 9.5920265e+04, 9.9850999e+04, 1.0359181e+05]
          }
O2p.electro = pd.DataFrame(O2p.electro)

O2p.electro # Print electro table in pandas format


In [ ]:
#NOp definition

NOp = specie()

NOp.thermo.M  = 30.0055514
NOp.thermo.A  = 3.02e-01
NOp.thermo.B  = -3.50
NOp.thermo.C  = -3.74
NOp.thermo.I  = 0
NOp.thermo.h  = 3.3000e+07
NOp.thermo.Ov = 2817
NOp.thermo.Or = 2.5
NOp.thermo.Ds = 3.49e+07
NOp.thermo.update()

NOp.electro = {'Level': [0, 1, 2, 3, 4,5 ,6 ,7], 
           'g': [1, 3, 6, 6, 3, 1, 2, 2],
           'Oe': [0, 7.5089678e+04, 8.5254624e+04, 
                 8.9035726e+04, 9.7469826e+04, 1.0005530e+05,
                 1.0280337e+05, 1.0571386e+05]
          }
NOp.electro = pd.DataFrame(NOp.electro)

NOp.electro # Print electro table in pandas format


In [ ]:
#Np definition

Np = specie()

Np.thermo.M  = 14.0061514
Np.thermo.A  = 1.16e-02
Np.thermo.B  = 6.03e-01
Np.thermo.C  = -1.24e+01
Np.thermo.I  = 0
Np.thermo.h  = 1.3438e+08
Np.thermo.Ov = 0
Np.thermo.Or = 0
Np.thermo.Ds = 0
Np.thermo.update()

Np.electro = {'Level': [0, 1, 2, 3, 4, 5 ,6], 
           'g': [1, 3, 5, 5, 1, 5, 15],
           'Oe': [0, 7.0068352e+01, 1.8819180e+02,
                  2.2036569e+04, 4.7031835e+04, 6.7312522e+04,
                  1.3271908e+05]
          }
Np.electro = pd.DataFrame(Np.electro)

Np.electro # Print electro table in pandas format


In [ ]:
#Op definition

Op = specie()

Op.thermo.M  = 15.9988514
Op.thermo.A  = 2.03e-02
Op.thermo.B  = 4.29e-01
Op.thermo.C  = -1.16e+01
Op.thermo.I  = 0
Op.thermo.h  = 9.8056e+07
Op.thermo.Ov = 0
Op.thermo.Or = 0
Op.thermo.Ds = 0
Op.thermo.update()

Op.electro = {'Level': [0, 1, 2], 
           'g': [4, 10,6],
           'Oe': [0, 3.8583347e+04, 5.8223492e+04]
          }
Op.electro = pd.DataFrame(Op.electro)

Op.electro # Print electro table in pandas format


In [ ]:
#em definition

em = specie()

em.thermo.M  = 0.00054858
em.thermo.A  = 0
em.thermo.B  = 0
em.thermo.C  = -1.20e+01
em.thermo.I  = 0
em.thermo.h  = 0
em.thermo.Ov = 0
em.thermo.Or = 0
em.thermo.Ds = 0
em.thermo.update()

em.electro = {'Level': [0], 
           'g': [1],
           'Oe': [0]
          }
em.electro = pd.DataFrame(em.electro)

em.electro # Print electro table in pandas format


In [ ]:
# Pandas usefull commands
# Show a column of the table
print(O2.electro['Oe'])

print('\n\n')
# Access a variable
print(' The requested variable is ' , O2.electro['Oe'][2])